# Assignment 4
## Description
datasets from [assets/wikipedia_data.html](assets/wikipedia_data.html). Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). 

**what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses.

Calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [107]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")

league='NHL'

def nhl_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    nhl_df=pd.read_csv("assets/nhl.csv")
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df['team'] = nhl_df['team'].str.replace(r'\*', "")
    nhl_df = nhl_df[['team', 'W', 'L']]
    
#     eliminate
    dropList=[]
    for i in range(nhl_df.shape[0]):
        row = nhl_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            dropList.append(i)
    nhl_df = nhl_df.drop(dropList)
    
    nhl_df['team'] = nhl_df['team'].str.replace('[\w.]* ', '')
    nhl_df = nhl_df.astype({'team': 'str', 'W': int, 'L': int})
    nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W']+nhl_df['L'])
    
    join = pd.merge(team, nhl_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    
    population_by_region = join['Population']
    win_loss_by_region = join['W/L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

# nhl_correlation()

Calculate Win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [109]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")



def nba_correlation():
    # YOUR CODE HERE
#     raise NotImplementedError()
    nba_df=pd.read_csv("assets/nba.csv")
    league='NBA'
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].str.replace(r'[\*]', "")
    nba_df['team'] = nba_df['team'].str.replace(r'\(\d*\)', "")
    nba_df['team'] = nba_df['team'].str.replace(r'[\xa0]', "")
    nba_df = nba_df[['team', 'W/L%']]
    nba_df['team'] = nba_df['team'].str.replace('[\w.]* ', '')
    nba_df = nba_df.astype({'team': str, 'W/L%': float})
    
    join = pd.merge(team, nba_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = join['Population'] 
    win_loss_by_region = join['W/L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
#     return nba_df
#     return np.isnan(population_by_region).any()
#     return np.isnan(win_loss_by_region).any()
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

# nba_correlation()

Calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [160]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")

def mlb_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    mlb_df=pd.read_csv("assets/mlb.csv")
    league='MLB'
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('\ Sox','Sox')
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'] = mlb_df['team'].str.replace(r'[\*]', "")
    mlb_df['team'] = mlb_df['team'].str.replace(r'\(\d*\)', "")
    mlb_df['team'] = mlb_df['team'].str.replace(r'[\xa0]', "")
    mlb_df = mlb_df[['team', 'W-L%']]
    mlb_df.rename(columns={'W-L%': 'W/L%'}, inplace=True)
    mlb_df['team']=mlb_df['team'].str.replace('\ Sox','Sox')
    mlb_df['team'] = mlb_df['team'].str.replace('[\w.]* ', '')
    
    mlb_df = mlb_df.astype({'team': str, 'W/L%': float})
    
    join = pd.merge(team, mlb_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = join['Population'] 
    win_loss_by_region = join['W/L%']
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

#     return mlb_df
#     return np.isnan(population_by_region).any()
#     return np.isnan(win_loss_by_region).any()

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15003737475409495

Calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [113]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")

def nfl_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    nfl_df=pd.read_csv("assets/nfl.csv")
    league='NFL'
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df['team'] = nfl_df['team'].str.replace(r'[\*]', "")
    nfl_df['team'] = nfl_df['team'].str.replace(r'\(\d*\)', "")
    nfl_df['team'] = nfl_df['team'].str.replace(r'[\xa0]', "")
    nfl_df.rename(columns={'W-L%': 'W/L%'}, inplace=True)
    nfl_df = nfl_df[['team', 'W/L%']]
    
#     eliminate
    dropList=[]
    for i in range(nfl_df.shape[0]):
        row = nfl_df.iloc[i]
        if row['team']==row['W/L%']:
            dropList.append(i)
    nfl_df = nfl_df.drop(dropList)
    
    nfl_df['team'] = nfl_df['team'].str.replace('[\w.]* ', '')
    nfl_df['team'] = nfl_df['team'].str.replace('+', '')
    nfl_df = nfl_df.astype({'team': 'str', 'W/L%': float})
    
    join = pd.merge(team, nfl_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    
    population_by_region = join['Population'] 
    win_loss_by_region = join['W/L%'] 
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

# nfl_correlation()

Explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**.

In [97]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")

def nhl_df(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    nhl_df=pd.read_csv("assets/nhl.csv")
    league='NHL'
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df['team'] = nhl_df['team'].str.replace(r'\*', "")
    nhl_df = nhl_df[['team', 'W', 'L']]
    
#     eliminate
    dropList=[]
    for i in range(nhl_df.shape[0]):
        row = nhl_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            dropList.append(i)
    nhl_df = nhl_df.drop(dropList)
    
    nhl_df['team'] = nhl_df['team'].str.replace('[\w.]* ', '')
    nhl_df = nhl_df.astype({'team': 'str', 'W': int, 'L': int})
    nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W']+nhl_df['L'])
    
    join = pd.merge(team, nhl_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    return join['W/L%']

def nba_df():
    # YOUR CODE HERE
#     raise NotImplementedError()
    nba_df=pd.read_csv("assets/nba.csv")
    league='NBA'
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].str.replace(r'[\*]', "")
    nba_df['team'] = nba_df['team'].str.replace(r'\(\d*\)', "")
    nba_df['team'] = nba_df['team'].str.replace(r'[\xa0]', "")
    nba_df = nba_df[['team', 'W/L%']]
    nba_df['team'] = nba_df['team'].str.replace('[\w.]* ', '')
    nba_df = nba_df.astype({'team': str, 'W/L%': float})
    
    join = pd.merge(team, nba_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = join['Population'] 
    win_loss_by_region = join['W/L%']
    return win_loss_by_region

def nfl_df(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    nfl_df=pd.read_csv("assets/nfl.csv")
    league='NFL'
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df['team'] = nfl_df['team'].str.replace(r'[\*]', "")
    nfl_df['team'] = nfl_df['team'].str.replace(r'\(\d*\)', "")
    nfl_df['team'] = nfl_df['team'].str.replace(r'[\xa0]', "")
    nfl_df.rename(columns={'W-L%': 'W/L%'}, inplace=True)
    nfl_df = nfl_df[['team', 'W/L%']]
    
#     eliminate
    dropList=[]
    for i in range(nfl_df.shape[0]):
        row = nfl_df.iloc[i]
        if row['team']==row['W/L%']:
            dropList.append(i)
    nfl_df = nfl_df.drop(dropList)
    
    nfl_df['team'] = nfl_df['team'].str.replace('[\w.]* ', '')
    nfl_df['team'] = nfl_df['team'].str.replace('+', '')
    nfl_df = nfl_df.astype({'team': 'str', 'W/L%': float})
    
    join = pd.merge(team, nfl_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    return join['W/L%']

def mlb_df(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    mlb_df=pd.read_csv("assets/mlb.csv")
    league='MLB'
    
    team = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("-", np.nan).dropna().reset_index().rename(columns = {'value': 'team'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ', '')
    
#     2018 stats
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'] = mlb_df['team'].str.replace(r'[\*]', "")
    mlb_df['team'] = mlb_df['team'].str.replace(r'\(\d*\)', "")
    mlb_df['team'] = mlb_df['team'].str.replace(r'[\xa0]', "")
    mlb_df.rename(columns={'W-L%': 'W/L%'}, inplace=True)
    mlb_df = mlb_df[['team', 'W/L%']]
    mlb_df['team'] = mlb_df['team'].str.replace('[\w.]* ', '')
    mlb_df = mlb_df.astype({'team': str, 'W/L%': float})
    
    join = pd.merge(team, mlb_df, 'outer', on='team')
    join = join.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    return join['W/L%']

def get_df(sport):
    if sport == 'NBA':
        return nba_df()
    elif sport == 'NHL':
        return nhl_df()
    elif sport == 'MLB':
        return mlb_df()
    elif sport == 'NFL':
        return nfl_df()
    return
    
def sports_team_performance():
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for x in sports:
        for y in sports:
            if x != y:
                join = pd.merge(get_df(x), get_df(y), how='inner', on=['Metropolitan area'])
                p_values.loc[x, y] = stats.ttest_rel(join['W/L%_x'], join['W/L%_y'])[1]
#                 print(join)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
#     assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.737840
NBA,0.937509,NaN,0.022386,0.860671
NHL,0.030318,0.022386,NaN,0.001090
MLB,0.737840,0.860671,0.001090,NaN
